# Computing the instanton of the Maier-Stein system

Ref: *R. S. Maier and D. L. Stein, Journal of Statistical Physics 83, 291 (1996)*

## Import statements

In [1]:
# Modify path environment variables to search for the local Pyritz and NLopt installations in the repository.

import os, sys

pyritz_dir = "/".join(os.getcwd().split("/")[:-1])
nlopt_lib_dir = "%s/nlopt/lib" % pyritz_dir
nlopt_py_dir = "%s/nlopt/nlopt_py" % pyritz_dir

if "LD_LIBRARY_PATH" in os.environ:
    paths = os.environ["LD_LIBRARY_PATH"].split(":")
    paths.append(nlopt_lib_dir)
    os.environ["LD_LIBRARY_PATH"] = ":".join(paths)
else:
    os.environ["LD_LIBRARY_PATH"] = ":%s" % nlopt_lib_dir
    
sys.path.insert(0, nlopt_py_dir)
sys.path.insert(0, pyritz_dir)

# Import statements

import pyritz

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import nlopt

## Define system

#### System:

$$
dX = a(X)dt + \sqrt{\epsilon} dW.
$$

where

$$
a(x_1, x_2) =
\begin{pmatrix}
x_1 - x_1^3 - \beta x_1 x_2^2 \\
-(1 + x_1^2) x_2
\end{pmatrix}
$$

and

$$
\nabla a = \begin{pmatrix}
    1 - 3 x_1^2 - \beta x_2^2 &  -2 x_1 x_2 \\
    - 2 \beta x_1 x_2 & - (1 + x_1^2)
\end{pmatrix}
$$

where $(\nabla a)_{ij} = \frac{\partial a_i}{\partial x_j}$.

#### Lagrangian:

$$
L(x, x') = |x'| |a(x)| - x' \cdot a(x)
$$

#### Derivatives of the Lagrangian:

$$
\begin{aligned}
\frac{\partial L}{\partial x_i} & = \sum_{j=1}^{\text{dim}} \left[ \nabla_i a_j( x ) \cdot \left( |\dot x| \hat{a}_j(x) - \dot x_j \right) \right] \\
\frac{\partial L}{\partial x'_i} & = |a(x)| \hat{x}'_i - a_i(x)
\end{aligned}
$$

In [ ]:
# System parameters

m_beta = 10

# Fixed points

e_xa = np.array([-1, 0])
e_xb = np.array([1,  0])
e_xs = np.array([0,  0])

# System Lagrangian

def system_a(xs):
    x1, x2 = xs
    return np.array([x1 - np.power(x1, 3) - m_beta * x1 * np.power(x2, 2), -(1 + np.power(x1, 2))*x2])

def lagrangian(ls, dxls, dvls, path, us, args):
    xs, vs = path
    
    sas = system_a(xs)
    v_norms = np.linalg.norm(vs, axis=0)
    sa_norms = np.linalg.norm(sas, axis=0)
    vs_dot_sas = np.einsum("ij,ij->j", vs, sas)
    
    ls[:] = v_norms * sa_norms - vs_dot_sas
    
    if not dxls is None:

        ## Compute dL/dx

        da = np.array([
            [1 - 3*xs[0,:]**2 - m_beta*xs[1,:]**2,     -2*xs[0,:]*xs[1,:]],
            [-2*m_beta*xs[0,:]*xs[1,:],                -(1+xs[0,:]**2)]
        ])

        f = np.where(sa_norms!=0)
        sa_units = np.zeros(sas.shape)
        sa_units[:, f] = sas[:, f]/sa_norms[f]

        dxls[:] = np.einsum("jki,ki->ji", da, v_norms*sa_units - vs)

        ## Compute dL/dv

        f = np.where(v_norms!=0)
        avs = np.zeros(vs.shape)
        avs[:, f] = vs[:, f]*sa_norms[f]/v_norms[f]

        dvls[:] = avs - sas

## Computing the instanton

### Gradient-based optimisation

In [ ]:
x1 = e_xa
x2 = e_xb

n = 8
nq = n*10

action = pyritz.interpolation.Action(lagrangian, n, nq, x1, x2)
alpha0 = pyritz.interpolation.utils.linear_path(x1, x2, n)
alpha0 += np.random.random(len(alpha0))*0.1 # Add random noise to the initial path

opt = nlopt.opt(nlopt.LD_SLSQP, np.size(alpha0))
opt.set_min_objective(action.compute)
opt.set_xtol_rel(1e-16)
alpha = opt.optimize(alpha0)

print("S[alpha0] = %s" % action.compute(alpha0))
print("S[alpha]  = %s" % action.compute(alpha))

#### Instanton plot

In [ ]:
us = np.linspace(-1, 1, 1000)

paths = [
    (action.get_alpha_with_endpoints(alpha0), "Initial"),
    (action.get_alpha_with_endpoints(alpha), "Final")
]

for _alpha, _label in paths:
    xs = pyritz.interpolation.utils.interpolate(_alpha, n, us)
    plt.plot(xs[0,:], xs[1,:], label=_label)

X, Y = np.meshgrid(np.linspace(-1.2,1.2,64), np.linspace(-0.4,.4,64))
sax,say=system_a([X,Y]); sax=sax/np.sqrt(sax**2+say**2); say=say/np.sqrt(sax**2+say**2)
plt.streamplot(X,Y, sax, say, density=1.7, linewidth=.6, color='gray');
plt.legend()

fig = mpl.pyplot.gcf()
fig.set_size_inches(7, 7)

### Gradient-free optimisation

In [ ]:
x1 = e_xa
x2 = e_xb

n = 8
nq = n*10

action = pyritz.interpolation.Action(lagrangian, n, nq, x1, x2)
alpha0 = pyritz.interpolation.utils.linear_path(x1, x2, n)
alpha0 += np.random.random(len(alpha0))*0.1 # Add random noise to the initial path

opt = nlopt.opt(nlopt.LN_NEWUOA, np.size(alpha0))
opt.set_min_objective(action.compute)
opt.set_xtol_rel(1e-16)
alpha = opt.optimize(alpha0)

print("S[alpha0] = %s" % action.compute(alpha0))
print("S[alpha]  = %s" % action.compute(alpha))

#### Instanton plot

In [ ]:
us = np.linspace(-1, 1, 1000)

paths = [
    (action.get_alpha_with_endpoints(alpha0), "Initial"),
    (action.get_alpha_with_endpoints(alpha), "Final")
]

for _alpha, _label in paths:
    xs = pyritz.interpolation.utils.interpolate(_alpha, n, us)
    plt.plot(xs[0,:], xs[1,:], label=_label)

X, Y = np.meshgrid(np.linspace(-1.2,1.2,64), np.linspace(-0.4,.4,64))
sax,say=system_a([X,Y]); sax=sax/np.sqrt(sax**2+say**2); say=say/np.sqrt(sax**2+say**2)
plt.streamplot(X,Y, sax, say, density=1.7, linewidth=.6, color='gray');
plt.legend()

fig = mpl.pyplot.gcf()
fig.set_size_inches(7, 7)